<a href="https://colab.research.google.com/github/dss5202-2410/Notebooks/blob/main/More_on_dplyr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# More on `dplyr`

`dplyr` is a library for the `R` language designed to make data analysis fast and easy.

In this section, we will introduce the `dfply` package, which makes it possible to do `dplyr`-style data manipulation with pipes in python on `pandas` DataFrames.

## Install and load package

In [45]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install dfply

`dfply` works directly on `pandas` DataFrames. It chains operations on data with the `>>` operator.

In [4]:
from dfply import *
diamonds >> head(3)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31


We can chain piped operations, and assign the output to a new DataFrame.

In [5]:
df1 = diamonds >> head(10)
df1

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
5,0.24,Very Good,J,VVS2,62.8,57.0,336,3.94,3.96,2.48
6,0.24,Very Good,I,VVS1,62.3,57.0,336,3.95,3.98,2.47
7,0.26,Very Good,H,SI1,61.9,55.0,337,4.07,4.11,2.53
8,0.22,Fair,E,VS2,65.1,61.0,337,3.87,3.78,2.49
9,0.23,Very Good,H,VS1,59.4,61.0,338,4.00,4.05,2.39


### The `X` DataFrame symbol

DataFrame passing through the pipe operation is represented by the symbol `X`. For example, the following code selects certain columns from the original DataFrame (`diamonds`).

In [6]:
diamonds >> select(X.carat, X.cut) >> head(10)

,carat,cut
0,0.23,Ideal
1,0.21,Premium
2,0.23,Good
3,0.29,Premium
4,0.31,Good
5,0.24,Very Good
6,0.24,Very Good
7,0.26,Very Good
8,0.22,Fair
9,0.23,Very Good


### Selecting and dropping variables

There are two functions for selection, `select` and `drop`. These functions accept string labels, integer positions, and/or symbolically represented column names (`X.column_name`).

In [7]:
diamonds >> select(["color", "clarity"], 1, X.carat, X.cut) >> head()

,color,clarity,cut,carat
0,E,SI2,Ideal,0.23
1,E,SI1,Premium,0.21
2,E,VS1,Good,0.23
3,I,VS2,Premium,0.29
4,J,SI2,Good,0.31


The `drop` function does the oposite. It returns all columns besides the ones specified.

In [8]:
diamonds >> drop(["color", "clarity"], 1, X.carat, X.cut) >> head()

,depth,table,price,x,y,z
0,61.5,55.0,326,3.95,3.98,2.43
1,59.8,61.0,326,3.89,3.84,2.31
2,56.9,65.0,327,4.05,4.07,2.31
3,62.4,58.0,334,4.20,4.23,2.63
4,63.3,58.0,335,4.34,4.35,2.75


One particularly nice thing about `dplyr` is that we can drop columns inside of a `select()` statement by putting a subtraction sign in front (`... %>% select(-col)`). This can also be done in `dfply`, with the tilde symbol (`~`).

For example, let's say we want to select any column except `carat` and `clarity`. One way to do this is to specify them for removal using the `~` operator.

In [10]:
diamonds >> select(~X.carat, ~X.clarity) >> head(2)

,cut,color,depth,table,price,x,y,z
0,Ideal,E,61.5,55.0,326,3.95,3.98,2.43
1,Premium,E,59.8,61.0,326,3.89,3.84,2.31


### Selection filter functions

+ `starts_with(prefix)`: Find columns that starts with a string prefix.

+ `ends_with(suffix)`: Find columns that ends with a string suffix.

+ `contains(string)`: Find columns that contain a string in their name.

+ `everything()`: All columns.

+ `columns_between(start_col, end_col, inclusive = True)`: Find columns between a specified start and end column. The `inclusive = True` boolean argument indicates whether the end column should be included or not.

+ `columns_to(end_col, inclusive = True)`: Get columns up to a specified end column.

+ `columns_from(start_col)`: Get columns starting at a specified column.

Let's see some examples.

Let's say we want to select only the columns that start with "c".

In [11]:
diamonds >> select(starts_with("c")) >> head(2)

,carat,cut,color,clarity
0,0.23,Ideal,E,SI2
1,0.21,Premium,E,SI1


The selection filter functions works with the inversion operator (`~`) too. Let's say we want to remove only the columns that start with "c".

In [12]:
diamonds >> select(~starts_with("c")) >> head(2)

,depth,table,price,x,y,z
0,61.5,55.0,326,3.95,3.98,2.43
1,59.8,61.0,326,3.89,3.84,2.31


They work the same inside the `drop` function. Let's say I want to remove all columns from `price` onwards.

In [13]:
diamonds >> drop(columns_from(X.price)) >> head(2)

,carat,cut,color,clarity,depth,table
0,0.23,Ideal,E,SI2,61.5,55.0
1,0.21,Premium,E,SI1,59.8,61.0


### Subsetting and filtering rows

Slice of rows can be done with `row_slice()`. You can pass single integer index or a list of indices to select rows.

The following code selects the 10-th and 15-th rows in `diamonds`.

In [14]:
diamonds >> row_slice([10, 15])

,carat,cut,color,clarity,depth,table,price,x,y,z
10,0.30,Good,J,SI1,64.0,55.0,339,4.25,4.28,2.73
15,0.32,Premium,E,I1,60.9,58.0,345,4.38,4.42,2.68


This can be used with `group_by()`, where we can select rows within sub-groups.

The following code selects the 5-th row for sub-groups in `cut`.

In [15]:
diamonds >> group_by("cut") >> row_slice(5)

,carat,cut,color,clarity,depth,table,price,x,y,z
128,0.91,Fair,H,SI2,64.4,57.0,2763,6.11,6.09,3.93
20,0.30,Good,I,SI2,63.3,56.0,351,4.26,4.30,2.71
40,0.33,Ideal,I,SI2,61.2,56.0,403,4.49,4.50,2.75
26,0.24,Premium,I,VS1,62.5,57.0,355,3.97,3.94,2.47
21,0.23,Very Good,E,VS2,63.8,55.0,352,3.85,3.92,2.48


The `sample()` function works exactly the same as `pandas`' `.sample()` method.

In [16]:
diamonds >> sample(frac = 0.0001, replace = False)

,carat,cut,color,clarity,depth,table,price,x,y,z
16684,0.31,Ideal,H,SI2,61.1,56.0,421,4.40,4.42,2.69
32714,0.31,Premium,F,VS2,62.0,58.0,802,4.37,4.31,2.69
21926,2.00,Good,G,SI2,64.2,60.0,9999,7.89,7.81,5.04
28381,0.33,Very Good,I,VS1,63.2,57.0,668,4.40,4.37,2.77
1340,0.72,Ideal,H,VS1,61.1,57.0,2961,5.80,5.82,3.55


In [17]:
diamonds >> sample(n = 3, replace = True)

,carat,cut,color,clarity,depth,table,price,x,y,z
30247,0.26,Ideal,E,VVS2,62.5,55.0,724,4.09,4.07,2.55
21466,1.55,Ideal,D,SI1,61.7,57.0,9565,7.47,7.43,4.59
30231,0.33,Ideal,E,VS2,62.3,57.0,723,4.40,4.43,2.75


The `distinct()` function selects unique rows. It is similar to the `.drop_duplicates()` method in `pandas`.

In [19]:
diamonds >> distinct(X.cut)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
5,0.24,Very Good,J,VVS2,62.8,57.0,336,3.94,3.96,2.48
8,0.22,Fair,E,VS2,65.1,61.0,337,3.87,3.78,2.49


Filtering rows with logical criteria is done with `mask()`, which accepts boolean statements and keep True-labeled rows.

In [20]:
diamonds >> mask(X.cut == "Ideal") >> head(4)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
11,0.23,Ideal,J,VS1,62.8,56.0,340,3.93,3.90,2.46
13,0.31,Ideal,J,SI2,62.2,54.0,344,4.35,4.37,2.71
16,0.30,Ideal,I,SI2,62.0,54.0,348,4.31,4.34,2.68


In [30]:
diamonds >> mask(X.cut == "Ideal", X.color == "E", X.price < 400)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
31595,0.20,Ideal,E,VS2,59.7,55.0,367,3.86,3.84,2.30
31599,0.20,Ideal,E,VS2,62.2,57.0,367,3.76,3.73,2.33


Alternatively, we can use `filter_by()`.

In [31]:
diamonds >> filter_by(X.cut == "Ideal", X.color == "E", X.price < 400)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
31595,0.20,Ideal,E,VS2,59.7,55.0,367,3.86,3.84,2.30
31599,0.20,Ideal,E,VS2,62.2,57.0,367,3.76,3.73,2.33


### DataFrame Transformation

New variables can be created with `mutate()`.

In [34]:
diamonds >> mutate(price_k = X.price/1000) >> head(2)

,carat,cut,color,clarity,depth,table,price,x,y,z,price_k
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,0.326
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,0.326


In [35]:
diamonds >> mutate(price_k = X.price/1000, x_plus_y = X.x + X.y) >> head(2)

,carat,cut,color,clarity,depth,table,price,x,y,z,price_k,x_plus_y
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,0.326,7.93
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,0.326,7.73


`transmute()` is a combination of `mutate()` and `select()` of the created variables.

In [36]:
diamonds >> transmute(price_k = X.price/1000, x_plus_y = X.x + X.y) >> head(2)

,price_k,x_plus_y
0,0.326,7.93
1,0.326,7.73


### Grouping

DataFrames can be grouped along variables using `group_by()` and ungrouped with `ungroup()`. Functions chained after grouping are applied by group until ungrouping.

In the example below, we compute the median diamond prices within each cut sub-group.

In [46]:
diamonds >> group_by(X.cut) >> mutate(med_price = median(X.price)) >> select(X.cut, X.med_price) >> row_slice(1) >> ungroup()

,cut,med_price
91,Fair,3282.0
4,Good,3050.5
11,Ideal,1810.0
3,Premium,3185.0
6,Very Good,2648.0


### Summarization

There are two summary functions in `dfply`: `summarize()` and `summarize_each()`.

The `summarize()` function computes summary statistics for specified columns in the DataFrame. It can also be used with groupings.

In [133]:
diamonds >> summarize(mean_price = X.price.mean(), std_price = X.price.std())

,price_mean,price_std
0,3932.799722,3989.439738


In [135]:
diamonds >> group_by(X.cut) >> summarize(mean_price = X.price.mean(), std_price = X.price.std(), med_price = X.price.median(),)

,cut,mean_price,std_price,med_price
0,Fair,4358.757764,3560.386612,3282.0
1,Good,3928.864452,3681.589584,3050.5
2,Ideal,3457.541970,3808.401172,1810.0
3,Premium,4584.257704,4349.204961,3185.0
4,Very Good,3981.759891,3935.862161,2648.0


The `summarize_each()` function is a more generalized function. It takes a list of summary functions to apply as its first argument, and then a list of columns to apply the summary functions to. It also works with groupings.

In [136]:
diamonds >> summarize_each([np.mean, np.std, np.median], X.price)

,price_mean,price_std,price_median
0,3932.799722,3989.402758,2401.0


In [137]:
diamonds >> group_by("cut") >> summarize_each([np.mean, np.std, np.median], X.price, X.carat)

,cut,price_mean,price_std,price_median,carat_mean,carat_std,carat_median
0,Fair,4358.757764,3559.280730,3282.0,1.046137,0.516244,1.00
1,Good,3928.864452,3681.214352,3050.5,0.849185,0.454008,0.82
2,Ideal,3457.541970,3808.312813,1810.0,0.702837,0.432866,0.54
3,Premium,4584.257704,4349.047276,3185.0,0.891955,0.515243,0.86
4,Very Good,3981.759891,3935.699276,2648.0,0.806381,0.459416,0.71


## Sorting, splitting, and uniting

Sorting is done by `arrange()`, which is similar to the `pandas`' `.sort_values()` method.

In [47]:
diamonds >> arrange(X.price, ascending = False) >> head(5)

,carat,cut,color,clarity,depth,table,price,x,y,z
27749,2.29,Premium,I,VS2,60.8,60.0,18823,8.50,8.47,5.16
27748,2.00,Very Good,G,SI1,63.5,56.0,18818,7.90,7.97,5.04
27747,1.51,Ideal,G,IF,61.7,55.0,18806,7.37,7.41,4.56
27746,2.07,Ideal,G,SI2,62.5,55.0,18804,8.20,8.13,5.11
27745,2.00,Very Good,H,SI1,62.8,57.0,18803,7.95,8.00,5.01


In [48]:
diamonds >> group_by(X.cut) >> arrange(X.price, ascending = False) >> head(3) >> ungroup()

,carat,cut,color,clarity,depth,table,price,x,y,z
27646,2.01,Fair,G,SI1,70.6,64.0,18574,7.43,6.64,4.69
27643,2.02,Fair,H,VS2,64.5,57.0,18565,8.00,7.95,5.14
27630,4.50,Fair,J,I1,65.8,58.0,18531,10.23,10.16,6.72
27739,2.80,Good,G,SI2,63.8,58.0,18788,8.90,8.85,0.00
27682,2.07,Good,I,VS2,61.8,61.0,18707,8.12,8.16,5.03
27672,2.67,Good,F,SI2,63.8,58.0,18686,8.69,8.64,5.54
27747,1.51,Ideal,G,IF,61.7,55.0,18806,7.37,7.41,4.56
27746,2.07,Ideal,G,SI2,62.5,55.0,18804,8.20,8.13,5.11
27741,2.15,Ideal,G,SI2,62.6,54.0,18791,8.29,8.35,5.21
27749,2.29,Premium,I,VS2,60.8,60.0,18823,8.50,8.47,5.16


The `rename()` function rename columns.

In [49]:
diamonds >> rename(CUT = X.cut, COLOR = X.color) >> head(2)

,carat,CUT,COLOR,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31


Columns can be split into multiple ones with `separate()`. It takes a variety of arguments:

+ `column`: The column to split.

+ `into`: The names of new columns.

+ `sep`: Either a regex string or integer positions to split the column on.

+ `remove`: Boolean indicating whether to remove the original column.

+ `convert`: Boolean indicating whether the new columns should be converted to the appropriate type.

In [63]:
# Create a DataFrame
df2 = pd.DataFrame({
  "var_date": ["2024-03-26", "2024-08-01"]
})
df2

,var_date
0,2024-03-26
1,2024-08-01


In [64]:
# Split column by the "-" separator
df3 = df2 >> separate(X.var_date, into = ["year", "month", "day"], sep = "-", remove = False, convert = True)
df3

,var_date,year,month,day
0,2024-03-26,2024,3,26
1,2024-08-01,2024,8,1


The `unite()` function does the opposite of `separate()`. It joins columns by a separator. The arguments of `unite()` are:

+ `colname`: The name of the new column.

+ `*args`: A list of columns to be joined.

+ `sep`: The string operator to join the columns with.

+ `remove`: Boolean indicating whether to remove the original columns.

In [65]:
df3 >> unite("new_date", X.year, X. month, X.day, sep = "/", remove = False)

['year', 'month', 'day'] / False maintain


,var_date,year,month,day,new_date
0,2024-03-26,2024,3,26,2024/3/26
1,2024-08-01,2024,8,1,2024/8/1


### Reshaping a DataFrame

A data frame can be presented in multiple ways. The example below shows the same data organized in four different ways, with each shows the same values of four variables: country, year, population, and TB cases.


In [112]:
table1 = pd.DataFrame({
  "country": ["Afghanistan", "Afghanistan", "Brazil", "Brazil", "China", "China"],
  "year": [1999, 2000, 1999, 2000, 1999, 2000],
  "cases": [745, 2666, 37737, 80488, 212258, 213766],
  "population": [19987071, 20595360, 172006362, 174504898, 1272915272, 1280428583]
})
table1

,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Afghanistan,2000,2666,20595360
2,Brazil,1999,37737,172006362
3,Brazil,2000,80488,174504898
4,China,1999,212258,1272915272
5,China,2000,213766,1280428583


In [101]:
table2 = pd.DataFrame({
  "country": ["Afghanistan", "Afghanistan", "Afghanistan", "Afghanistan", "Brazil", "Brazil", "Brazil", "Brazil", "China", "China", "China", "China"],
  "year": [1999, 1999, 2000, 2000, 1999, 1999, 2000, 2000, 1999, 1999, 2000, 2000],
  "type": ["cases", "population", "cases", "population", "cases", "population", "cases", "population", "cases", "population", "cases", "population"],
  "val": [745, 19987071, 2666, 20595360, 37737, 172006362, 80488, 174504898, 212258, 1272915272, 213766, 1280428583]
})
table2

,country,year,type,val
0,Afghanistan,1999,cases,745
1,Afghanistan,1999,population,19987071
2,Afghanistan,2000,cases,2666
3,Afghanistan,2000,population,20595360
4,Brazil,1999,cases,37737
5,Brazil,1999,population,172006362
6,Brazil,2000,cases,80488
7,Brazil,2000,population,174504898
8,China,1999,cases,212258
9,China,1999,population,1272915272


In [72]:
table3 = pd.DataFrame({
  "country": ["Afghanistan", "Brazil", "China"],
  "TB_1999": [745, 37737, 212258],
  "TB_2000": [2666, 80488, 213766]
})
table3

,country,TB_1999,TB_2000
0,Afghanistan,745,2666
1,Brazil,37737,80488
2,China,212258,213766


These are all representations of the same underlying data, but they are not equally easy to use.

Only `table1` is tidy. Recall the three interrelated rules of tidy data:

1. Each variable must have its own column.

2. Each observation must have its own row.

3. Each value must have its own cell.

<img src="https://d33wubrfki0l68.cloudfront.net/6f1ddb544fc5c69a2478e444ab8112fb0eea23f8/91adc/images/tidy-1.png" width="600">

There are two main advantages of tidy data:

1. There's a general advantage to picking one consistent way of storing data. If you have a consistent data structure, it's easier to learn the tools that work with it because they have an underlying uniformity.

2. There's a specific advantage to placing variables in columns. As you learned in `mutate()` and `group_by()`, most functions work with vectors of values. That makes transforming tidy data feel particularly natural.

A common problem of a data set is that column names are not names of variables, but values of a variable -- that's the problem with `table3`.

+ The column names `TB_1999` and `TB_2000` represent values of a `year` variable, the values in these two columns represent values of a `TBcases` variable. Each row represent two observations, not one.

To tidy up a dataset like this, we need the `gather()` function. The syntax is `gather(key, value, *columns)`, where

+ `key` is the name of the new column to store the variable (in this case, `year`). Since this variable does not exist in the original table, we put its name in quotes.

+ `value` is the name of the new column to store the value (in this case, `TBcases`). Since this variable does not exist in the original table, we put its name in quotes.

+ `*columns` are a list of variables whose names are values (in this case, `TB_1999` and `TB_2000`).

In [75]:
table3 >> gather("year", "TBcases", ["TB_1999", "TB_2000"])

,country,year,TBcases
0,Afghanistan,TB_1999,745
1,Brazil,TB_1999,37737
2,China,TB_1999,212258
3,Afghanistan,TB_2000,2666
4,Brazil,TB_2000,80488
5,China,TB_2000,213766


We can further `separate()` the values in the `year` column, and only keep the values for the years.

The resulting DataFrame is tidy, it contains the TB cases in each country in each year.

In [81]:
(table3
 >> gather("year", "TBcases", ["TB_1999", "TB_2000"])
 >> separate(X.year, into = ["temp", "year"], sep = "_", remove = False, convert = True)
 >> select(~X.temp))

,country,year,TBcases
0,Afghanistan,1999,745
1,Brazil,1999,37737
2,China,1999,212258
3,Afghanistan,2000,2666
4,Brazil,2000,80488
5,China,2000,213766


Another problem that a DataFrame can have is that an obsevation is scattered across multiple rows. Take `table2` as an example: An observation is a country in a year, but each observation is spread across two rows.

To tidy it up, we will use the `spread()` function. The basic syntax is `spread(key, values)`.

+ `key` is the column to take variable names from (in this example, `type`).

+ `values` is the column to take values from (in this case, `count`).

+ `convert` accepts a boolean indicating whether to convert the new variables into appropriate types.

In [105]:
table2 >> spread(X.type, X.val, convert = True) >> rename(TBcases = "cases")

,country,year,TBcases,population
0,Afghanistan,1999,745,19987071
1,Afghanistan,2000,2666,20595360
2,Brazil,1999,37737,172006362
3,Brazil,2000,80488,174504898
4,China,1999,212258,1272915272
5,China,2000,213766,1280428583


Now, the table is back in a tidy format.

### Joining tables

It's rare that a data analysis involves only a single table of data. Typically you have many tables of data, and you must combine them to answer the questions that you're interested in. Collectively, multiple tables of data are called relational data because it is the relations, not just the individual datasets, that are important.

Relations are always defined between a pair of tables. All other relations are built up from this simple idea: the relations of three or more tables are always a property of the relations between each pair. Sometimes both elements of a pair can be the same table! This is needed if, for example, you have a table of people, and each person has a reference to their parents.

To work with relational data you need verbs that work with pairs of tables. There are three families of verbs designed to work with relational data:

+ Mutating joins, which add new variables to one data frame from matching observations in another.

    + `inner_join()`

    + `left_join()`

    + `right_join()`

    + `outer_join()`

+ Filtering joins, which filter observations from one data frame based on whether or not they match an observation in the other table.

    + `semi_join()`

    + `anti_join()`

The functionaliries of them will be outlined with the toy example below:

<img src="https://d33wubrfki0l68.cloudfront.net/108c0749d084c03103f8e1e8276c20e06357b124/5f113/diagrams/join-setup.png" width="200">

In [120]:
x = pd.DataFrame({
        "key": [1, 2, 3],
        "value_x": ["x1", "x2", "x3"]
    })
y = pd.DataFrame({
    "key": [1, 2, 4],
    "value_y": ["y1", "y2", "y3"]
})

The simplest type of join is an `inner_join()`. It matches pairs of observations whenevery their keys are equal.

<img src="https://d33wubrfki0l68.cloudfront.net/3abea0b730526c3f053a3838953c35a0ccbe8980/7f29b/diagrams/join-inner.png" width="450">

The most important property of `inner_join()` is that the unmatched rows are not included in the result.

In [121]:
x >> inner_join(y, by = "key")

,key,value_x,value_y
0,1,x1,y1
1,2,x2,y2


While `inner_join()` keeps observations that appear in both tables, an outer join keeps observations that appear in at least one of the tables. There are three types of outer joins:

+ `left_join()`: Keeps all observations in `x`.

+ `right_join()`: Keeps all observations in `y`.

+ `outer_join()`: Keeps all observations in both `x` and `y`.

These work by adding an additional "virtual" observation to each table. This observation has a key that always matches and a value filled with `NaN`.

In [122]:
x >> left_join(y, by = "key")

,key,value_x,value_y
0,1,x1,y1
1,2,x2,y2
2,3,x3,NaN


In [123]:
x >> right_join(y, by = "key")

,key,value_x,value_y
0,1,x1,y1
1,2,x2,y2
2,4,NaN,y3


In [124]:
x >> outer_join(y, by = "key")

,key,value_x,value_y
0,1,x1,y1
1,2,x2,y2
2,3,x3,NaN
3,4,NaN,y3


Filtering joins match observations in the same way as mutating joins, but only affect the observatons, instead of variables.

+ `semi_join()` keeps all observations in `x` that have a match in `y`.

+ `anti_join()` drops all observations in `x` that have a match in `y`. In other words, it only keeps the ones without any match.

In [125]:
x >> semi_join(y, by = "key")

,key,value_x
0,1,x1
1,2,x2


In [126]:
x >> anti_join(y, by = "key")

,key,value_x
2,3,x3


### Building DataFrames

We can also join DataFrames by rows or by columns.

+ `bind_rows()` concatinates two DataFrames "vertically".

+ `bind_cols()` joins DataFrames "horizontally".

In [128]:
x >> bind_rows(y)

,key,value_x,value_y
0,1,x1,NaN
1,2,x2,NaN
2,3,x3,NaN
0,1,NaN,y1
1,2,NaN,y2
2,4,NaN,y3


In [132]:
x >> bind_cols(y >> select(~X.key))

,key,value_x,value_y
0,1,x1,y1
1,2,x2,y2
2,3,x3,y3
